### Nonemployer Statistics API Tutorial
The goal in creating this notebook is to give data users who are interested in data related to the the Milwaukee-Waukesha, WI MSA, the Milwaukee-Racine-Waukesha, WI CSA and Milwaukee County a guide to Nonemployer Statisics (NES) data. This notebook will show a user how to:

* Build the base URL for any Census Bureau API request
* Define arguments that can be used in a NES API query and build an example NES query
* Create a Python `class` or separate functions that create NES tables with data related to Milwaukee

With these objectives defined, let's get started.

#### Building a Census Bureau API request
Any request to the Census API has a few basic elements in the base URL:

```
https://api.census.gov/data/**year**/**program name**
```

First, the base URL for any dataset is always `https://api.census.gov/data`. If you want data for a specific year, such as 2018, add that to the year argument. Finally, if you want NES data for any year from 1997 to 2018, specify `nonemp` in the program name argument. 

So, if you wanted NES data for Milwaukee County for 2018

```
https://api.census.gov/data/2018/nonemp
```

#### Variables
There are a few variables that users should know about for Nonemployer Statistics data:

| Variable Name | Description | Variable Type | Required | Additional Attributes |
| :------------ | :---------- | :------------ | :------- | :-------------------- |
| `COUNTY` | County name/code? | Geography | No; not a predicate | None |
| `CSA` | Combined Statistical Area code/name?; <br> available 2017, 2018 | Geography | No; not a predicate | None |
| `NESTAB` | Number of establishments for specified <br> sector-geography combination | Numeric | No | `ESTAB_N`: establishment flag |
| `for` | Census API FIPS '`for`' clause | FIPS-for | predicate only | None |
| `NAME`/`GEO_TTL` | Geographic Area Name; additional attribute <br> of `GEO_ID`, geographic identifier code. | string | No | None |
| `in` | Census API FIPS '`in`' clause | FIPS-in | predicate only | None |
| `LFO` | Legal Form of Organization code, only <br> available at state and national level; <br> avaiable 2008-2018 | string | No | `LFO_LABEL` or `LFO_TTL`: the label for <br> the LFO code |
| `NAICSYYYY` | YYYY NAICS code (industry code) | string | Default displayed | `NAICSYYYY_LABEL` or `NAICSYYYY_TTL` : the label associated <br> with the `NAICSYYYY` code |
| `NRCPTOT` | Nonemployer sales, value of shipments, <br> or revenue (1,000s of dollars) | numeric | No | `NRCPTOT_F`: flag for `RCPTOT` |
| `RCPSZES` | Sales, value of shipments, or <br> revenue size of establishment code | string | default displayed | `RCPSZES_LABEL` or `RCPSZES_TTL`: label associated <br> with `RCPSZES` code | 
| `YEAR` | Year associated with CBP data <br> queried with the API | numeric | No | None |

Additional variables available at <a href="https://api.census.gov/data/2018/nonemp/variables.html">Nonemployer Statistics API Variables page</a>. For specific years, change 2018 in the web address to a different year (1997-2018).

#### Building a full query
A few extra notes before building an example NES query.

* You **can** iterate through a list of geographies with your API query. However, NES queries can only contain lists of counties within the same state if you are trying to query for multiple counties.
* You **cannot** iterate through lists of LFO codes, RCPSZES codes, NAICS codes, etc. The list of these codes cannot be added to your predicate, but specific combinations can be added to your predicate. I will add code that allows users to iterate through combinations of LFO-RCPSZES-NAICS codes.

With that, let's start building code to query for Milwaukee-related NES data.

In [1]:
import os
import json
import sys
import requests
import pandas as pd
import numpy as np

class milwaukee_nes_api:
    def __init__(self, request_url, data = None):
        self.request_url = request_url
        self.data = data
        
    def load_api_key(path = "census_api.txt"):
        """ 
        This is meant to load the API key needed to request Census data.
        """
        try:
            with open(path, 'r') as f:
                # It's assumed our file contains a single line,
                # with our API key
                return f.read().strip()
        except FileNotFoundError:
            print("'%s' file not found" % path)
            
    def mke_county_nes_data(year, api_key = None, save_path = None):
        """
        mke_county_nes_data is a method that allows a user to get the Nonemployer Statistics 
        data for years 1998 to 2018 (TO-DO: add method to get NES data 1997). Right now, this
        method returns all available NES data with these columns:
        
        - RCPSZES: Sales, valuee of shipments, or revenue size of establishment code;
                   a table of these codes and the corresponding RCPSZES_LABEL/RCPSZES_TTL
                   codes are provided in the pseudo-markdown table below:
                   
        # | RCPSZES | RCPSZES_LABEL/RCPSZES_TTL |
        # | :------ | :------------------------ |
        # | 001" | "All establishments" |
        # | "111" | "Establishments with sales, value of shipments, or revenue less than $5,000" |
        # | "113" | "Establishments with sales, value of shipments, or revenue less than $25,000" |
        # | "1131" | "Establishments with sales, value of shipments, or revenue less than $50,000" |
        # | "114" | "Establishments with sales, value of shipments, or revenue less than $100,000" |
        # | "118" | "Establishments with sales, value of shipments, or revenue of $5,000 to $9,999" |
        # | "119" | "Establishments with sales, value of shipments, or revenue of $10,000 to $24,999" |
        # | "1191" | "Establishments with sales, value of shipments, or revenue of $25,000 to $29,999" |
        # | "121" | "Establishments with sales, value of shipments, or revenue of $25,000 to $49,999" |
        # | "1211" | "Establishments with sales, value of shipments, or revenue of $30,000 to $49,999" |
        # | "122" | "Establishments with sales, value of shipments, or revenue of $50,000 to $99,999" |
        # | "123" | "Establishments with sales, value of shipments, or revenue of $100,000 to $249,999" |
        # | "1241" | "Establishments with sales, value of shipments, or revenue of $100,000 to $199,999" |
        # | "1242" | "Establishments with sales, value of shipments, or revenue of $100,000 to $249,999" |
        # | "1243" | "Establishments with sales, value of shipments, or revenue of $200,000 to $249,999" |
        # | "1244" | "Establishments with sales, value of shipments, or revenue of $250,000 to $299,999" |
        # | "125" | "Establishments with sales, value of shipments, or revenue of $250,000 to $499,999" |
        # | "1251" | "Establishments with sales, value of shipments, or revenue of $300,000 to $499,999" |
        # | "131" | "Establishments with sales, value of shipments, or revenue of $500,000 to $999,999" |
        # | "132" | "Establishments with sales, value of shipments, or revenue of $1,000,000 or more" |
        # | "1321" | "Establishments with sales, value of shipments, or revenue of $1,000,000 to $1,999,999" |
        # | "133" | "Establishments with sales, value of shipments, or revenue of $1,000,000 to $2,499,999" |
        # | "1331" | "Establishments with sales, value of shipments, or revenue of $2,000,000 to $2,499,999" |
        # | "141" | "Establishments with sales, value of shipments, or revenue of $2,500,000 to $4,999,999" |
        # | "1411" | "Establishments with sales, value of shipments, or revenue of $5,000,000 or more" |
        # | "142" | "Establishments with sales, value of shipments, or revenue of $5,000,000 to $9,999,999" |
        # | "143" | "Establishments with sales, value of shipments, or revenue of $10,000,000 or more" |
        
        - NESTAB: the number of nonemployer establishments
        - NRCPTOT: the total of nonemployer receipts, i.e. nonemployer sales, value of 
                   shipments, or revenue ($1,000)
        - NAICSYYYY: YYYY NAICS code; valid for years 1998-present;
                     NAICSYYYY_LABEL/NAICSYYYY_TTL is the corresponding label for each
                     NAICS code. For more information, visit this page:
                     https://api.census.gov/data/2018/nonemp/variables/NAICS2017.json
                     
                     YYYY = 1997, 2002, 2007, 2012, or 2017
        - YEAR: Reference year for NES data request.
        
        Input:
        
            * year: an int; should be the year you want data for
            * api_key: a string; this should be your Census Bureau API key. If
                       you do not have an API key from the Census Bureau, you can
                       request one at: https://api.census.gov/data/key_signup.html
                       You can put in API requests to api.census.gov without a key
                       (keep api_key=None), but you will be rate limited to 500 
                       requests per day
            * save_path: a string; this should be the path directory + name of the
                         file you would like to save your file to. Currently, this
                         method only saves to Excel, CSV (and JSON?) files
        
        Output: a table with all NES data for Milwaukee County, WI for a specified
                reference year
        """
        # Make sure data is cast to the right type of data for request
        if api_key is None:
            api_arg = ""
        else:
            api_arg = f'&key={api_key}'
        # Set base URL
        base_url = f'https://api.census.gov/data/{year}/nonemp'
        # If data requested is from 2017-2018 (and most likely 2017-2021 once NES data becomes available)
        if int(year) in range(2017, 2019):
            # TO-DO: add method to let user add/remove columns
            columns = ["NAME", "YEAR", "NAICS2017", "NAICS2017_LABEL", "RCPSZES", "RCPSZES_LABEL",
                       "NESTAB", "NRCPTOT"]
            # Combine columns for query
            comb_cols = ",".join(map(str, columns))
        # Case when year is in 2012-2016
        elif int(year) in range(2012, 2017):
            # TO-DO: add method to let user add/remove columns
            columns = ["NAME", "YEAR", "NAICS2012", "NAICS2012_TTL", "RCPSZES", "RCPSZES_TTL",
                       "NESTAB", "NRCPTOT"]
            # Combine columns for query
            comb_cols = ",".join(map(str, columns))
        # Case where year is in 2008-2011
        elif int(year) in range(2008, 2012):
            # TO-DO: add method to let user add/remove columns
            columns = ["GEO_TTL", "NAICS2007", "NAICS2007_TTL", "RCPSZES", "RCPSZES_TTL",
                       "NESTAB", "NRCPTOT"]
            # Combine columns for query
            comb_cols = ",".join(map(str, columns))
        # Case where year is in 2003-2007
        elif int(year) in range(2003, 2008):
            # TO-DO: add method to let user add/remove columns
            columns = ["GEO_TTL", "NAICS2002", "NAICS_TTL", "RCPSZES", "RCPSZES_TTL",
                       "NESTAB", "NRCPTOT"]
            # Combine columns for query
            comb_cols = ",".join(map(str, columns))
        elif int(year) in range(1998, 2003):
            # TO-DO: add method to let user add/remove columns
            columns = ["GEO_TTL", "YEAR", "NAICS1997", "NAICS_TTL", "RCPSZES", "RCPSZES_TTL",
                       "NESTAB", "NRCPTOT"]
            # Combine columns for query
            comb_cols = ",".join(map(str, columns))
        else:
            print(str(year) + " not currently available for NES API requests.")
            return None
        # Create data URL, Milwaukee County, WI FIPS codes are CTY_FIPS = 079 & ST_FIPS = 55
        data_url = f'{base_url}?get={comb_cols}&for=county:079&in=state:55{api_arg}'
        #print(data_url)
        # Get response from Census API
        response = requests.get(data_url)
        #print(response.status_code)
        data = response.json()
        # Sort data by NAICS and RCPSZES codes
        if int(year) in range(2017, 2019):
            mke_county_nes = pd.DataFrame(data[1:], columns=data[0]).iloc[:, :-2].sort_values(by=["NAICS2017", "RCPSZES"],
                                                                                              ascending=True).reset_index(drop=True)
        elif int(year) in range(2012, 2017):
            mke_county_nes = pd.DataFrame(data[1:], columns=data[0]).iloc[:, :-2].sort_values(by=["NAICS2012", "RCPSZES"],
                                                                                              ascending=True).reset_index(drop=True)
        elif int(year) in range(2008, 2012):
            mke_county_nes = pd.DataFrame(data[1:], columns=data[0]).iloc[:, :-2].sort_values(by=["NAICS2007", "RCPSZES"],
                                                                                              ascending=True).reset_index(drop=True)
        elif int(year) in range(2003, 2008):
            mke_county_nes = pd.DataFrame(data[1:], columns=data[0]).iloc[:, :-2].sort_values(by=["NAICS2002", "RCPSZES"],
                                                                                              ascending=True).reset_index(drop=True)
        elif int(year) in range(1998, 2003):
            mke_county_nes = pd.DataFrame(data[1:], columns=data[0]).iloc[:, :-2].sort_values(by=["NAICS1997", "RCPSZES"],
                                                                                              ascending=True).reset_index(drop=True)
        # Convert numeric data from strings to np.int64 types and replace 0s with NaNs (which
        # is what the data should be)
        mke_county_nes["NESTAB"] = mke_county_nes.NESTAB.astype(np.int64).replace(0, np.nan)
        mke_county_nes["NRCPTOT"] = mke_county_nes.NRCPTOT.astype(np.int64).replace(0, np.nan)
        mke_county_nes["YEAR"] = mke_county_nes.YEAR.astype(int)
        # Process of handling save_path argument
        if save_path is not None:
            if str(save_path).split(".")[1] == "csv":
                mke_county_nes.to_csv(str(save_path))
            elif str(save_path).split(".") == "xlsx":
                mke_county_nes.to_excel(str(save_path))
            else:
                print("Save path invalid or file type not currently supported.")
        return mke_county_nes
    
    def mke_waukesha_west_allis_msa_nes(year, api_key = None, save_path = None):
        """
        mke_waukesha_west_allis_msa_nes is a method that allows a user to get the Nonemployer
        Statistics data for years 2016 to 2018. Right now, this method returns all available NES
        data with these columns:
        
        - RCPSZES: Sales, valuee of shipments, or revenue size of establishment code;
                   a table of these codes and the corresponding RCPSZES_LABEL/RCPSZES_TTL
                   codes are provided in the pseudo-markdown table below:
                   
        # | RCPSZES | RCPSZES_LABEL/RCPSZES_TTL |
        # | :------ | :------------------------ |
        # | 001" | "All establishments" |
        # | "111" | "Establishments with sales, value of shipments, or revenue less than $5,000" |
        # | "113" | "Establishments with sales, value of shipments, or revenue less than $25,000" |
        # | "1131" | "Establishments with sales, value of shipments, or revenue less than $50,000" |
        # | "114" | "Establishments with sales, value of shipments, or revenue less than $100,000" |
        # | "118" | "Establishments with sales, value of shipments, or revenue of $5,000 to $9,999" |
        # | "119" | "Establishments with sales, value of shipments, or revenue of $10,000 to $24,999" |
        # | "1191" | "Establishments with sales, value of shipments, or revenue of $25,000 to $29,999" |
        # | "121" | "Establishments with sales, value of shipments, or revenue of $25,000 to $49,999" |
        # | "1211" | "Establishments with sales, value of shipments, or revenue of $30,000 to $49,999" |
        # | "122" | "Establishments with sales, value of shipments, or revenue of $50,000 to $99,999" |
        # | "123" | "Establishments with sales, value of shipments, or revenue of $100,000 to $249,999" |
        # | "1241" | "Establishments with sales, value of shipments, or revenue of $100,000 to $199,999" |
        # | "1242" | "Establishments with sales, value of shipments, or revenue of $100,000 to $249,999" |
        # | "1243" | "Establishments with sales, value of shipments, or revenue of $200,000 to $249,999" |
        # | "1244" | "Establishments with sales, value of shipments, or revenue of $250,000 to $299,999" |
        # | "125" | "Establishments with sales, value of shipments, or revenue of $250,000 to $499,999" |
        # | "1251" | "Establishments with sales, value of shipments, or revenue of $300,000 to $499,999" |
        # | "131" | "Establishments with sales, value of shipments, or revenue of $500,000 to $999,999" |
        # | "132" | "Establishments with sales, value of shipments, or revenue of $1,000,000 or more" |
        # | "1321" | "Establishments with sales, value of shipments, or revenue of $1,000,000 to $1,999,999" |
        # | "133" | "Establishments with sales, value of shipments, or revenue of $1,000,000 to $2,499,999" |
        # | "1331" | "Establishments with sales, value of shipments, or revenue of $2,000,000 to $2,499,999" |
        # | "141" | "Establishments with sales, value of shipments, or revenue of $2,500,000 to $4,999,999" |
        # | "1411" | "Establishments with sales, value of shipments, or revenue of $5,000,000 or more" |
        # | "142" | "Establishments with sales, value of shipments, or revenue of $5,000,000 to $9,999,999" |
        # | "143" | "Establishments with sales, value of shipments, or revenue of $10,000,000 or more" |
        
        - NESTAB: the number of nonemployer establishments
        - NRCPTOT: the total of nonemployer receipts, i.e. nonemployer sales, value of 
                   shipments, or revenue ($1,000)
        - NAICSYYYY: YYYY NAICS code; valid for years 1998-present;
                     NAICSYYYY_LABEL/NAICSYYYY_TTL is the corresponding label for each
                     NAICS code. For more information, visit this page:
                     https://api.census.gov/data/2018/nonemp/variables/NAICS2017.json
                     
                     YYYY = 2017
        - YEAR: Reference year for NES data request.
        
        Input:
        
            * year: an int; should be the year you want data for
            * api_key: a string; this should be your Census Bureau API key. If
                       you do not have an API key from the Census Bureau, you can
                       request one at: https://api.census.gov/data/key_signup.html
                       You can put in API requests to api.census.gov without a key,
                       (keep api_key=None) but you will be rate limited to 500 
                       requests per day
            * save_path: a string; this should be the path directory + name of the
                         file you would like to save your file to. Currently, this
                         method only saves to Excel, CSV (and JSON?) files
        
        Output: a table with all NES data for Milwaukee-Waukesha-West Allis, WI MSA for a specified
                reference year
        """
        # Make sure data is cast to the right type of data for request
        if api_key is None:
            api_arg = ""
        else:
            api_arg = f'&key={api_key}'
        # Set base URL
        base_url = f'https://api.census.gov/data/{year}/nonemp'
        # If data requested is from 2017-2018
        if int(year)in range(2017, 2019):
            # TO-DO: add method to let user add/remove columns
            columns = ["NAME", "YEAR", "NAICS2017", "NAICS2017_LABEL", "RCPSZES", "RCPSZES_LABEL",
                       "NESTAB", "NRCPTOT"]
            # Combine columns for query
            comb_cols = ",".join(map(str, columns))
        # Casw where request is for 2016
        elif int(year) == 2016:
            # TO-DO: add method to let user add/remove columns
            columns = ["NAME", "YEAR", "NAICS2012", "NAICS2012_TTL", "RCPSZES", "RCPSZES_TTL",
                       "NESTAB", "NRCPTOT"]
            # Combine columns for query
            comb_cols = ",".join(map(str, columns))
        # Print NES data unavailable for entered reference year & return None
        else:
            print(str(year) + " not currently available for NES API requests.")
            return None
        # Create data URL, Milwaukee MSA code is 33340
        data_url = f'{base_url}?get={comb_cols}&for=metropolitan%20statistical%20area/micropolitan%20statistical%20area:33340{api_arg}'
        #print(data_url)
        # Get response from Census API
        response = requests.get(data_url)
        #print(response.status_code)
        data = response.json()
        # Sort data by NAICS and RCPSZES codes
        if int(year) in range(2017, 2019):
            mke_msa_nes = pd.DataFrame(data[1:], columns=data[0]).iloc[:, :-1].sort_values(by=["NAICS2017", "RCPSZES"],
                                                                                              ascending=True).reset_index(drop=True)
        elif int(year) == 2016:
            mke_msa_nes = pd.DataFrame(data[1:], columns=data[0]).iloc[:, :-1].sort_values(by=["NAICS2012", "RCPSZES"],
                                                                                              ascending=True).reset_index(drop=True)
        # Convert numeric data from strings to np.int64 types and replace 0s with NaNs (which
        # is what the data should be)
        mke_msa_nes["NESTAB"] = mke_msa_nes.NESTAB.astype(np.int64).replace(0, np.nan)
        mke_msa_nes["NRCPTOT"] = mke_msa_nes.NRCPTOT.astype(np.int64).replace(0, np.nan)
        mke_msa_nes["YEAR"] = mke_msa_nes.YEAR.astype(int)
        # Process of handling save_path argument
        if save_path is not None:
            if str(save_path).split(".")[1] == "csv":
                mke_msa_nes.to_csv(str(save_path))
            elif str(save_path).split(".") == "xlsx":
                mke_msa_nes.to_excel(str(save_path))
            else:
                print("Save path invalid or file type not currently supported.")
        return mke_msa_nes
    
    def mke_csa_nes(year, api_key = None, save_path = None):
        """
        mke_csa_nes is a method that allows a user to get the Nonemployer Statistics data
        for years 2016 to 2018 for Milwaukee-Racine-Waukesha, WI CSA. Right now, this method
        returns all available NES data with these columns:
        
        - RCPSZES: Sales, valuee of shipments, or revenue size of establishment code;
                   a table of these codes and the corresponding RCPSZES_LABEL/RCPSZES_TTL
                   codes are provided in the pseudo-markdown table below:
                   
        # | RCPSZES | RCPSZES_LABEL/RCPSZES_TTL |
        # | :------ | :------------------------ |
        # | 001" | "All establishments" |
        # | "111" | "Establishments with sales, value of shipments, or revenue less than $5,000" |
        # | "113" | "Establishments with sales, value of shipments, or revenue less than $25,000" |
        # | "1131" | "Establishments with sales, value of shipments, or revenue less than $50,000" |
        # | "114" | "Establishments with sales, value of shipments, or revenue less than $100,000" |
        # | "118" | "Establishments with sales, value of shipments, or revenue of $5,000 to $9,999" |
        # | "119" | "Establishments with sales, value of shipments, or revenue of $10,000 to $24,999" |
        # | "1191" | "Establishments with sales, value of shipments, or revenue of $25,000 to $29,999" |
        # | "121" | "Establishments with sales, value of shipments, or revenue of $25,000 to $49,999" |
        # | "1211" | "Establishments with sales, value of shipments, or revenue of $30,000 to $49,999" |
        # | "122" | "Establishments with sales, value of shipments, or revenue of $50,000 to $99,999" |
        # | "123" | "Establishments with sales, value of shipments, or revenue of $100,000 to $249,999" |
        # | "1241" | "Establishments with sales, value of shipments, or revenue of $100,000 to $199,999" |
        # | "1242" | "Establishments with sales, value of shipments, or revenue of $100,000 to $249,999" |
        # | "1243" | "Establishments with sales, value of shipments, or revenue of $200,000 to $249,999" |
        # | "1244" | "Establishments with sales, value of shipments, or revenue of $250,000 to $299,999" |
        # | "125" | "Establishments with sales, value of shipments, or revenue of $250,000 to $499,999" |
        # | "1251" | "Establishments with sales, value of shipments, or revenue of $300,000 to $499,999" |
        # | "131" | "Establishments with sales, value of shipments, or revenue of $500,000 to $999,999" |
        # | "132" | "Establishments with sales, value of shipments, or revenue of $1,000,000 or more" |
        # | "1321" | "Establishments with sales, value of shipments, or revenue of $1,000,000 to $1,999,999" |
        # | "133" | "Establishments with sales, value of shipments, or revenue of $1,000,000 to $2,499,999" |
        # | "1331" | "Establishments with sales, value of shipments, or revenue of $2,000,000 to $2,499,999" |
        # | "141" | "Establishments with sales, value of shipments, or revenue of $2,500,000 to $4,999,999" |
        # | "1411" | "Establishments with sales, value of shipments, or revenue of $5,000,000 or more" |
        # | "142" | "Establishments with sales, value of shipments, or revenue of $5,000,000 to $9,999,999" |
        # | "143" | "Establishments with sales, value of shipments, or revenue of $10,000,000 or more" |
        
        - NESTAB: the number of nonemployer establishments
        - NRCPTOT: the total of nonemployer receipts, i.e. nonemployer sales, value of 
                   shipments, or revenue ($1,000)
        - NAICSYYYY: YYYY NAICS code; valid for years 1998-present;
                     NAICSYYYY_LABEL/NAICSYYYY_TTL is the corresponding label for each
                     NAICS code. For more information, visit this page:
                     https://api.census.gov/data/2018/nonemp/variables/NAICS2017.json
                     
                     YYYY = 2017
        - YEAR: Reference year for NES data request.
        
        Input:
        
            * year: an int; should be the year you want data for
            * api_key: a string; this should be your Census Bureau API key. If
                       you do not have an API key from the Census Bureau, you can
                       request one at: https://api.census.gov/data/key_signup.html
                       You can put in API requests to api.census.gov without a key,
                       (keep api_key=None) but you will be rate limited to 500 
                       requests per day
            * save_path: a string; this should be the path directory + name of the
                         file you would like to save your file to. Currently, this
                         method only saves to Excel, CSV (and JSON?) files
        
        Output: a table with all NES data for Milwaukee-Racine-Waukesha WI CSA for a specified
                reference year
        """
        # Make sure data is cast to the right type of data for request
        if api_key is None:
            api_arg = ""
        else:
            api_arg = f'&key={api_key}'
        # Set base URL
        base_url = f'https://api.census.gov/data/{year}/nonemp'
        # If data requested is from 2017-2018
        if int(year)in range(2017, 2019):
            # TO-DO: add method to let user add/remove columns
            columns = ["NAME", "YEAR", "NAICS2017", "NAICS2017_LABEL", "RCPSZES", "RCPSZES_LABEL",
                       "NESTAB", "NRCPTOT"]
            # Combine columns for query
            comb_cols = ",".join(map(str, columns))
        # Casw where request is for 2016
        elif int(year) == 2016:
            # TO-DO: add method to let user add/remove columns
            columns = ["NAME", "YEAR", "NAICS2012", "NAICS2012_TTL", "RCPSZES", "RCPSZES_TTL",
                       "NESTAB", "NRCPTOT"]
            # Combine columns for query
            comb_cols = ",".join(map(str, columns))
            # Combine columns for query
            comb_cols = ",".join(map(str, columns))
        # Print NES data unavailable for entered reference year & return None
        else:
            print(str(year) + " not currently available for NES API requests.")
            return None
        # Create data URL, Milwaukee-Racine-Waukesha, WI CSA code is 376
        data_url = f'{base_url}?get={comb_cols}&for=combined%20statistical%20area:376{api_arg}'
        #print(data_url)
        # Get response from Census API
        response = requests.get(data_url)
        #print(response.status_code)
        data = response.json()
        # Sort data by NAICS and RCPSZES codes
        if int(year) in range(2017, 2019):
            mke_csa_nes = pd.DataFrame(data[1:], columns=data[0]).iloc[:, :-1].sort_values(by=["NAICS2017", "RCPSZES"],
                                                                                              ascending=True).reset_index(drop=True)
        elif int(year) == 2016:
            mke_csa_nes = pd.DataFrame(data[1:], columns=data[0]).iloc[:, :-1].sort_values(by=["NAICS2012", "RCPSZES"],
                                                                                              ascending=True).reset_index(drop=True)
        # Convert numeric data from strings to np.int64 types and replace 0s with NaNs (which
        # is what the data should be)
        mke_csa_nes["NESTAB"] = mke_csa_nes.NESTAB.astype(np.int64).replace(0, np.nan)
        mke_csa_nes["NRCPTOT"] = mke_csa_nes.NRCPTOT.astype(np.int64).replace(0, np.nan)
        mke_csa_nes["YEAR"] = mke_csa_nes.YEAR.astype(int)
        # Process of handling save_path argument
        if save_path is not None:
            if str(save_path).split(".")[1] == "csv":
                mke_csa_nes.to_csv(str(save_path))
            elif str(save_path).split(".") == "xlsx":
                mke_csa_nes.to_excel(str(save_path))
            else:
                print("Save path invalid or file type not currently supported.")
        return mke_csa_nes

#### Examples of code execution
I will show a couple of examples of how some of the methods in the `milwaukee_nes_api` class work.

In [2]:
api_key = milwaukee_nes_api.load_api_key()

In [3]:
milwaukee_nes_api.mke_county_nes_data(year=2018, api_key=api_key)

,NAME,YEAR,NAICS2017,NAICS2017_LABEL,RCPSZES,RCPSZES_LABEL,NESTAB,NRCPTOT
0,"Milwaukee County, Wisconsin",2018,00,Total for all sectors,001,All establishments,50963,2193836
1,"Milwaukee County, Wisconsin",2018,11,"Agriculture, forestry, fishing and hunting",001,All establishments,71,2066
2,"Milwaukee County, Wisconsin",2018,113,Forestry and logging,001,All establishments,14,169
3,"Milwaukee County, Wisconsin",2018,114,"Fishing, hunting and trapping",001,All establishments,20,495
4,"Milwaukee County, Wisconsin",2018,1141,Fishing,001,All establishments,16,280
...,...,...,...,...,...,...,...,...
414,"Milwaukee County, Wisconsin",2018,81291,Pet care (except veterinary) services,001,All establishments,178,2977
415,"Milwaukee County, Wisconsin",2018,81292,Photofinishing,001,All establishments,10,157
416,"Milwaukee County, Wisconsin",2018,81293,Parking lots and garages,001,All establishments,9,4527
417,"Milwaukee County, Wisconsin",2018,81299,All other personal services,001,All establishments,1144,26061


In [4]:
milwaukee_nes_api.mke_csa_nes(year=2018, api_key=api_key)

,NAME,YEAR,NAICS2017,NAICS2017_LABEL,RCPSZES,RCPSZES_LABEL,NESTAB,NRCPTOT
0,"Milwaukee-Racine-Waukesha, WI CSA",2018,00,Total for all sectors,001,All establishments,121663,5928123
1,"Milwaukee-Racine-Waukesha, WI CSA",2018,11,"Agriculture, forestry, fishing and hunting",001,All establishments,668,21412
2,"Milwaukee-Racine-Waukesha, WI CSA",2018,113,Forestry and logging,001,All establishments,94,2411
3,"Milwaukee-Racine-Waukesha, WI CSA",2018,114,"Fishing, hunting and trapping",001,All establishments,126,2465
4,"Milwaukee-Racine-Waukesha, WI CSA",2018,1141,Fishing,001,All establishments,68,1125
...,...,...,...,...,...,...,...,...
445,"Milwaukee-Racine-Waukesha, WI CSA",2018,81291,Pet care (except veterinary) services,001,All establishments,515,11233
446,"Milwaukee-Racine-Waukesha, WI CSA",2018,81292,Photofinishing,001,All establishments,23,295
447,"Milwaukee-Racine-Waukesha, WI CSA",2018,81293,Parking lots and garages,001,All establishments,19,4629
448,"Milwaukee-Racine-Waukesha, WI CSA",2018,81299,All other personal services,001,All establishments,2747,73456


In [5]:
milwaukee_nes_api.mke_waukesha_west_allis_msa_nes(year=2018, api_key=api_key)

,NAME,YEAR,NAICS2017,NAICS2017_LABEL,RCPSZES,RCPSZES_LABEL,NESTAB,NRCPTOT
0,"Milwaukee-Waukesha-West Allis, WI Metro Area",2018,00,Total for all sectors,001,All establishments,95058,4666963
1,"Milwaukee-Waukesha-West Allis, WI Metro Area",2018,11,"Agriculture, forestry, fishing and hunting",001,All establishments,319,9655
2,"Milwaukee-Waukesha-West Allis, WI Metro Area",2018,113,Forestry and logging,001,All establishments,59,1348
3,"Milwaukee-Waukesha-West Allis, WI Metro Area",2018,114,"Fishing, hunting and trapping",001,All establishments,74,1437
4,"Milwaukee-Waukesha-West Allis, WI Metro Area",2018,1141,Fishing,001,All establishments,41,634
...,...,...,...,...,...,...,...,...
442,"Milwaukee-Waukesha-West Allis, WI Metro Area",2018,81291,Pet care (except veterinary) services,001,All establishments,376,7453
443,"Milwaukee-Waukesha-West Allis, WI Metro Area",2018,81292,Photofinishing,001,All establishments,17,261
444,"Milwaukee-Waukesha-West Allis, WI Metro Area",2018,81293,Parking lots and garages,001,All establishments,13,4591
445,"Milwaukee-Waukesha-West Allis, WI Metro Area",2018,81299,All other personal services,001,All establishments,2080,52271
